Intrucat cele 92 de modele de contracte gasite pe https://legeaz.net/modele au fost date in format pdf, le-am transformat printr-un script in format .txt

In [ ]:
import fitz
import os
from tqdm import tqdm

input_folder = "contracte_pdf_2"
output_folder = "contracte_txt_2"
os.makedirs(output_folder, exist_ok=True)

for file in tqdm(os.listdir(input_folder)):
    if file.endswith(".pdf"):
        pdf_path = os.path.join(input_folder, file)
        txt_path = os.path.join(output_folder, file.replace(".pdf", ".txt"))

        text = ""
        with fitz.open(pdf_path) as doc:
            for page in doc:
                text += page.get_text("text")

        with open(txt_path, "w", encoding="utf-8") as f:
            f.write(text)


In [ ]:
Pentru cele 29 de contracte gasite pe https://notari.pro, le-am transformat din .docx in .txt

In [ ]:
import os
from docx import Document
from tqdm import tqdm

input_folder = "notari.ro modele"
output_folder = "contracte_txt_2"
os.makedirs(output_folder, exist_ok=True)

for file in tqdm(os.listdir(input_folder)):
    if file.endswith(".docx"):
        doc_path = os.path.join(input_folder, file)
        txt_path = os.path.join(output_folder, file.replace(".docx", ".txt"))

        doc = Document(doc_path)
        text = "\n".join(p.text for p in doc.paragraphs)

        with open(txt_path, "w", encoding="utf-8") as f:
            f.write(text)


Dupa crearea unui folder cu documentele .txt, le-am formatat drept perechi intrebare-raspuns, in functie de titlul documentului.
ex: titlu = Model contract vanzare-cumparare
    intrebare: Genereaza un contract de vanzare-cumparare.

In [ ]:
import os
from tqdm import tqdm

input_folder = "contracte_txt"
output_folder = "intrebare-raspuns"
os.makedirs(output_folder, exist_ok=True)

for file in tqdm(os.listdir(input_folder)):
    if file.endswith(".txt"):
        input_path = os.path.join(input_folder, file)
        output_path = os.path.join(output_folder, file)

        with open(input_path, "r", encoding="utf-8") as f:
            lines = [line.strip() for line in f.readlines() if line.strip()]

        if not lines:
            continue

        title = lines[0]
        answer = "\n".join(lines[1:]).strip()

        question = f"Generează un {title.lower()}"

        formatted = (
            f"<|startoftext|>\n"
            f"Întrebare: {question}\n"
            f"Răspuns: {answer}\n"
            f"<|endoftext|>"
        )

        with open(output_path, "w", encoding="utf-8") as f:
            f.write(formatted)


Am strans toate datele din fisiere diferite intr-un singur txt

In [ ]:
import pandas as pd
import glob

all_texts = []
for path in glob.glob("intrebare-raspuns/*.txt"):
    with open(path, encoding="utf-8") as f:
        all_texts.append(f.read())

with open("dataset.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(all_texts))


Am trasnpus datele obtinute intr-un JSON pentru a putea fi date ulterior modelului.

In [ ]:
import re
import json

input_file = "dataset.txt"
output_file = "dataset.json"

with open(input_file, "r", encoding="utf-8") as f:
    content = f.read()

blocks = re.findall(r"<\|startoftext\|>(.*?)<\|endoftext\|>", content, re.DOTALL)

data = []
for block in blocks:
    q_match = re.search(r"Întrebare:\s*(.*)", block)
    a_match = re.search(r"Răspuns:\s*(.*)", block, re.DOTALL)

    if q_match and a_match:
        question = q_match.group(1).strip()
        answer = a_match.group(1).strip()
        data.append({"question": question, "answer": answer})

with open(output_file, "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)
